##MAX CUT Problem

Part A: Adjacency Matrix

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip -q install pyomo
from pyomo.environ import *
!apt-get install -y -qq coinor-cbc
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')

In [2]:
#PLEASE NOTE THAT THE GRAPH GIVEN HAS NODES AS A,B,C ...., I am reindexing them as numbers.

indexofnodes = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5}
nodesofindex = {0:'A', 1: 'B', 2: 'C', 3: 'D', 4:'E', 5: 'F'}

In [3]:
#creation of adjacency matrix
n = len(indexofnodes)
#using visual inspection to create a connections list manually.
adjacencymatrix = np.zeros((n,n))

#here each row in adjacency list of first element of the array.
connections = [[0,2,1,5,4], [1,2,0,4,3],[2,0,1,3,4,5],[3,1,2,5],[4,0,1,2,5],[5,0,2,3,4]]
for nodeconnection in connections:
  i = nodeconnection[0]
  for j in range(1,len(nodeconnection)):
    adjacencymatrix[i][nodeconnection[j]] = 1

#aij represents connection from i to j
adjdf = pd.DataFrame(adjacencymatrix, index=[nodesofindex[i] for i in range(n)], columns = [nodesofindex[i] for i in range(n)])
adjdf = adjdf.astype(int)
adjdf

,A,B,C,D,E,F
A,0,1,1,0,1,1
B,1,0,1,1,1,0
C,1,1,0,1,1,1
D,0,1,1,0,0,1
E,1,1,1,0,0,1
F,1,0,1,1,1,0


PART B: Identifying cut

In [4]:
#creating weight matrix, wij, where wij represents weights between i and j node.
weightlist = [[0,2,16],[0,5,36],[0,1,46],[0,4,3],[5,2,7],[4,2,3],[4,5,31],[4,1,15],[1,2,20],[1,3,46],[2,3,20],[3,5,19]]
weightmatrix = np.zeros((n,n))
for weight in weightlist:
  i,j,w = weight
  weightmatrix[i][j] = w
  weightmatrix[j][i] = w
weightmatrix = weightmatrix.astype(int)
wdf = pd.DataFrame(weightmatrix, index=[nodesofindex[i] for i in range(n)], columns = [nodesofindex[i] for i in range(n)])
wdf

,A,B,C,D,E,F
A,0,46,16,0,3,36
B,46,0,20,46,15,0
C,16,20,0,20,3,7
D,0,46,20,0,0,19
E,3,15,3,0,0,31
F,36,0,7,19,31,0


In [5]:
adjdf.iloc[0,2]

1

In [7]:
## creating pyomo model of the given formulation
rowindices = [i for i in range(n)]
colindices = [i for i in range(n)]
model = ConcreteModel()

#both e and x are {0,1}
model.e = Var(rowindices,colindices, domain=Binary)
model.x = Var(rowindices, domain = Binary)

#maximizing sum of weights*eij
model.objective = Objective(expr = sum(wdf.iloc[i,j]*model.e[i,j] for i in rowindices for j in colindices),sense = maximize)

model.constraints = ConstraintList()
for i in rowindices:
  for j in colindices:
    #adding constraints for the nodes that are connected.
    if adjdf.iloc[i,j] == 1:
      model.constraints.add(expr = model.e[i,j] <= model.x[i]+model.x[j])
      model.constraints.add(expr = model.e[i,j] <= 2- model.x[i]-model.x[j])
    else: model.constraints.add(expr = model.e[i,j] == 0)
    #doing this because if nodes are not connected then, it will go uninitialized, that's
    #why i am initializing it to 0, since we are maximizing, it's contribution will be nothing

model.pprint()

5 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   60 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60}
    e_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : e_index_0*e_index_1 :   36 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5)}
    e_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 

In [8]:
results = optCBC.solve(model)

In [9]:
#objective value
model.objective()

464.0

In [10]:
#printing the cut results
for i in rowindices:
  print(f'Node {nodesofindex[i]} belongs to set: {model.x[i].value}')

Node A belongs to set: 0.0
Node B belongs to set: 1.0
Node C belongs to set: 1.0
Node D belongs to set: 0.0
Node E belongs to set: 0.0
Node F belongs to set: 1.0


using the information above we can conclude that maxcut set is:

**S** : A, D, E


**V/S** : B, C, F